In [2]:
import requests
import pandas as pd

In [4]:
response = requests.get('https://api.data.codeup.com/api/v1/stores')
data = response.json()
stores = pd.DataFrame(data['payload']['stores'])

In [16]:
stores = pd.DataFrame(stores)

2 Approaches:

- loop until we don't have a next page
- make 1 request and figure out the max page, then loop over all the pages

In [18]:
# Approach 1
domain = 'https://api.data.codeup.com'
endpoint = '/api/v1/items'
items = []

while True:
    url = domain + endpoint
    response = requests.get(url)
    data = response.json()
    print(f'\rGetting page {data["payload"]["page"]} of {data["payload"]["max_page"]}: {url}', end='')
    items.extend(data['payload']['items'])
    endpoint = data['payload']['next_page']
    if endpoint is None:
        break

Getting page 3 of 3: https://api.data.codeup.com/api/v1/items?page=3

In [11]:
items = pd.DataFrame(items)

In [12]:
# Approach 2
base_url = 'https://api.data.codeup.com/api/v1/sales?page='
sales = []

# make the first request
url = base_url + str(1)
response = requests.get(url)
data = response.json()
max_page = data['payload']['max_page']
sales.extend(data['payload']['sales'])

# We already made the request to the first page, so we'll start at 2.
# We add 1 to max_page because the range() function is exclusive of the endpoint
page_range = range(2, max_page + 1)

for page in page_range:
    url = base_url + str(page)
    print(f'\rFetching page {page}/{max_page} {url}', end='')
    response = requests.get(url)
    data = response.json()
    sales.extend(data['payload']['sales'])

183/183 https://api.data.codeup.com/api/v1/sales?page=183

In [13]:
sales = pd.DataFrame(sales)

In [17]:
stores.to_csv('stores.csv', index=False)
items.to_csv('items.csv', index=False)
sales.to_csv('sales.csv', index=False)